In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Libraries

from pathlib import Path
import shutil
from bs4 import BeautifulSoup
import os

# Automatic Number Plate Reconigtion (ANPR)
1. Survey the dataset (image, annotations)
2. Combine multiple dataset to create a large number plate dataset for training data
3. Train Yolo model to recognize the correct bounding box the each car in an image
4. Use EasyOCR to extract number from car plate. Used OpenCV to visualize extracted result.
5. Apply ANPR on a video and save results to file

## 1. Survey the dataset
### Take a look at a sample. 
Load an image to their labels. I have drawn the image bounding box in green and its center in red

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt

img = cv.imread('/kaggle/input/car-plate-detection/images/Cars0.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

rec = cv.rectangle(img, (226,125), (419,173), (0,250,0),2)
rec = cv.circle(rec, ((226+419)//2,(125+173)//2), 2, (255,0,0),2)
plt.imshow(rec)

## 2. Create Dataset

Since I am going to use Yolo model. I need to do some data preparation before training

Some requirements for data:

**1. dataset.yaml**

```python

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../datasets/coco128  # dataset root dir  
train: images/train2017  # (relative to 'path')
val: images/train2017  # (relative to 'path')  
test:  # (optional)
    
# Classes
names:  
  0: person  
  1: bicycle  
  2: car  
  ...
```

**2. label.txt**

One *.txt file per image (if no objects in image, no *.txt file is required). The *.txt file specifications are:
* One row per object
* Each row have the following format (seperate by space):  ```class x_center y_center width height```
* Box coordinates (xywh) must be normalized (between 0 and 1). If your boxes are in pixels, divide ```x_center``` and ```width``` by ```image width```, and ```y_center``` and ```height``` by ```image height```.
* Class numbers are zero-indexed (start from 0)

<img src="https://user-images.githubusercontent.com/26833433/112467037-d2568c00-8d66-11eb-8796-55402ac0d62f.png" alt= "label example" width=400 height=200>

**3. Directories organization**

<img src="https://user-images.githubusercontent.com/26833433/134436012-65111ad1-9541-4853-81a6-f19a3468b75f.png" alt= "directories example" width=600 height=500>

In [ ]:
def normalized_coordinates(filename, width, height, xmin, ymin, xmax, ymax):
    """Take in image coordinates (unnormalized) as input, return normalized values 
    """
    
    xmin, xmax = xmin / width, xmax / width
    ymin, ymax = ymin / height, ymax/ height

    width = xmax-xmin
    height = ymax-ymin
    x_center = xmin + (width / 2)
    y_center = ymin + (height / 2)

    return x_center, y_center, width, height

def write_label(filename, x_center, y_center, width, height):
    """Save image's coordinates in text file named "filename"
    """
    with open(filename, mode='w') as outf:
        outf.write(f"{0} {x_center} {y_center} {width} {height}\n")

def parse_xml_tags(data):
    """Parse xml label file, return image file name, and its coordinates as a dictionary
    """
    tags = ['filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']
    Bs_data = BeautifulSoup(data, "xml")
    d = dict()

    for t in tags:
        text = Bs_data.find(t).text
        if all(c.isdigit() for c in text):
            d[t] = int(text)
        else:
            d[t] = text
    return d

def build_data(dir_folder, ann_file_list, img_dir):
    """Write xml labels to text file with specifications format, save at 'labels' folder.
        Move image to 'images' folder
    """
    images_folder = f"{dir_folder}/images"
    labels_folder = f"{dir_folder}/labels"
    
    os.makedirs(images_folder, exist_ok = True)
    os.makedirs(labels_folder, exist_ok = True)


    for ann_file in ann_file_list:
        with open(ann_file, 'r') as f:
            label = parse_xml_tags(f.read())
         
        img_file_name = label['filename']
        x_center, y_center, width, height = normalized_coordinates(**label)
         
        # save at 'labels' folder
        write_label(f"{labels_folder}/{img_file_name.split('.')[0]}.txt", x_center, y_center, width, height)
         
         # Move image to 'images' folder
        shutil.copy(f"{img_dir}/{img_file_name}", f"{images_folder}/{img_file_name}")

In [ ]:
import glob
dir_folder = "/kaggle/working/plate_datasets"

ann_list = glob.glob('/kaggle/input/car-plate-detection/annotations/*')
build_data(dir_folder, ann_list, "/kaggle/input/car-plate-detection/images")

ann_list = glob.glob('/kaggle/input/number-plate-detection/images/*.xml')
build_data(dir_folder, ann_list, "/kaggle/input/number-plate-detection/images")

In [ ]:
import yaml

data = {
    "path": dir_folder,
    "train": "images",
    "val": "",
    "names": {0: "car_lisence_plate"}
    }

with open('/kaggle/working/plate_datasets/dataset.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
    
# path: /content/plate_data  # dataset root dir
# train: /content/plate_data/train/images  # (relative to 'path')
# val: /content/plate_data/val/images  # (relative to 'path')
# test:  # test images (optional)

# names:
#   0: no_car_plate
#   1: car_lisence_plate
#   2: car



In [ ]:
assert len(os.listdir("/kaggle/working/plate_datasets/labels")) == len(os.listdir("/kaggle/working/plate_datasets/images"))

## 3. Train plate recognition model 
### Install model

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt

### Train model

In [ ]:
!python train.py --img 640 --batch 16 --epochs 15 --data /kaggle/working/plate_datasets/dataset.yaml --weights yolov5s.pt --cache ram

### Load trained model

In [ ]:
import torch

yolo = torch.hub.load('ultralytics/yolov5', 'custom', path='/kaggle/working/yolov5/runs/train/exp/weights/best.pt')

### Inference

In [ ]:
img = '/kaggle/input/number-plate-detection/TEST/TEST.jpeg'

# Inference
results = yolo(img)

cordinates = results.xyxy[0][:, :-1]
results.pandas().xyxy[0]

## 4. Extract Number Plate

### Install EasyOCR

In [ ]:
!pip install easyocr -qq

### Read Plate Number

In [2]:
def read_plate_number(results, frame, reader):
    n = len(results)
    x_shape, y_shape = frame.shape[1], frame.shape[0]

    for i in range(n):
        row = cordinates[i] # Iterate through each image
        if row[4] >= 0.5: ## Take img with 0.5 confidence
            xmin, ymin, xmax, ymax = row[:4]
            plate = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

            ## Preprocess Plate
            gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)
            blurred = cv.bilateralFilter(gray, 17, 15, 15)
#             ret, thresh = cv.threshold(blurred, 125, 255, cv.THRESH_BINARY)

            ## OCR
            text = reader.readtext(blurred)
            text = ' '.join([t[1] for t in text])

            plot_img = frame.copy()

            cv.rectangle(plot_img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2) ## BBox
            cv.rectangle(plot_img, (int(xmin), int(ymin-20)), (int(xmax), int(ymin)), (0, 255,0), -1) ## for text label background
            final_img = cv.putText(plot_img, f"{text}", (int(xmin), int(ymin)), cv.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 2)
            
            return final_img

#             return cv.cvtColor(final_img, cv.COLOR_BGR2RGB)
#             cv.imwrite(f'/kaggle/working/labeled_img_{i}.jpg', cv.cvtColor(final_img, cv.COLOR_BGR2RGB))

In [ ]:
import matplotlib.pyplot as plt
import easyocr
%matplotlib inline

reader = easyocr.Reader(['en'])
# cordinates = cordinates.cpu().numpy()

frame = cv.imread(img) ### reading the image
frame = cv.cvtColor(frame,cv.COLOR_BGR2RGB)

result_img = read_plate_number(cordinates, frame, reader)
plt.imshow(result_img)
plt.show();